In [ ]:
class CustomDatabase:
    """
    Class describing the Custom DriveU Dataset containing a list of images
    Attributes:
        images (List of DriveuImage)  All images of the dataset
        file_path (string):           Path of the dataset (.json)
    """

    def __init__(self, images):
        self.images = []
        self.input_images = images

    def open(self, data_base_dir: str = ""):
        """
        Method loading the dataset
        Args:
            data_base_dir(str): Base path where images are stored, optional
            if image paths in json are outdated
        """
        
        for image_dict in self.input_images:
            # parse and store image
            image = DriveuImage()
            image.parse_image_dict(image_dict, data_base_dir)
            self.images.append(image)

        return True    

In [ ]:
import argparse
import logging
import sys

import numpy as np

import cv2
from dtld_parsing.calibration import CalibrationData
# from dtld_parsing.driveu_dataset import DriveuDatabase
import dtld_parsing.driveu_dataset

from dtld_parsing.three_dimensional_position import ThreeDimensionalPosition


def positions_3D(args):
    """
    Function from dtld_parsing repo which calculates the position of the 
    objects with respect to the camera frame in meters.
    """
    # Load database
    if args.images:
        DriveuDatabase = CustomDatabase        
        database = DriveuDatabase(args.images)
    else:
        DriveuDatabase = dtld_parsing.driveu_dataset.DriveuDatabase
        database = DriveuDatabase(args.label_file)
        
    if not database.open(args.data_base_dir):
        return False

    # Load calibration
    calibration_left = CalibrationData()
    intrinsic_left = calibration_left.load_intrinsic_matrix(
        args.calib_dir + "/intrinsic_left.yml"
    )
    rectification_left = calibration_left.load_rectification_matrix(
        args.calib_dir + "/rectification_left.yml"
    )
    projection_left = calibration_left.load_projection_matrix(
        args.calib_dir + "/projection_left.yml"
    )
    extrinsic = calibration_left.load_extrinsic_matrix(
        args.calib_dir + "/extrinsic.yml"
    )
    distortion_left = calibration_left.load_distortion_matrix(
        args.calib_dir + "/distortion_left.yml"
    )
    calibration_right = CalibrationData()
    projection_right = calibration_right.load_projection_matrix(
        args.calib_dir + "/projection_right.yml"
    )

    threed_position = ThreeDimensionalPosition(
        calibration_left=calibration_left,
        calibration_right=calibration_right,
        binning_x=2,
        binning_y=2,
        roi_offset_x=0,
        roi_offset_y=0,
    )
    
    TL_coords = []
    class_list = []
    no_pred_id = []
    
    for idx_d, img in enumerate(database.images):       
#         rects = img.map_labels_to_disparity_image(calibration_left)
#         if img.file_path.split('/')[-2] == '2015-04-17_11-11-46':
#             print(img.file_path)
        try:
            disparity_image = img.get_disparity_image()
        except:
            print('Reached the limit')
            break
        #         import pdb;pdb.set_trace()
        
        TL_per_img = []
        classes_img = []
        # Get 3D position
        for o in img.objects:    
            aspect = o.attributes['aspects']
            direction = o.attributes['direction']
            orientation = o.attributes['orientation']
            occ = o.attributes['occlusion']
            reflection = o.attributes['reflection']
            x =  o.x
            y =  o.y
            w =  o.width
            h =  o.height
            if x < 0:
                w = w + x
                x = 0                       
            elif y < 0:
                h = h + y
                y = 0
            elif x + w > 2048:
                w = 2048 - x
            elif y + h > 1024:
                h = 1024 - y 
            
            o.x = x
            o.y = y
            o.width = w
            o.height = h        
            if aspect == '':
                TL_per_img.append([[0, 0]])
                continue
            
            if reflection == 'reflected' or occ == 'occluded' or aspect == 'four_aspects' \
                    or orientation == 'horizontal' or aspect == 'unknown':                
                continue

            # Camera Coordinate System: X right, Y down, Z front
            threed_pos = threed_position.determine_three_dimensional_position(
                o.x, o.y, o.width, o.height, disparity_image=disparity_image
            )
            # Get in vehicle coordinates: X front, Y left and Z up
            threed_pos_numpy = np.array([threed_pos.get_pos()[0], 
                                         threed_pos.get_pos()[1], 
                                         threed_pos.get_pos()[2], 
                                         1])
            threed_pos_vehicle_coordinates = extrinsic.dot(threed_pos_numpy)
            classes = [aspect, direction]
            TL_per_img.append(threed_pos_vehicle_coordinates[:2])   # take only x, y
            classes_img.append(classes)
        
        if len(TL_per_img):   
            TL_coords.append(np.array(TL_per_img))
            class_list.append(classes_img)
        else:
            TL_coords.append(np.array([[0, 0]]))
            class_list.append([['', '']])
        
    return TL_coords, class_list


In [ ]:
import json
import itertools
import utm
import numpy as np
import osrm
import folium
import glob
from folium import plugins    
from math import cos, asin, sqrt, pi


def get_GPS(json_path):
    """
    From DTLD v2.0 label file, GPS measurements are taken and 
    grouped by sequences.
    Returns grouped longitude latitude list, velocity and yaw
    rate list, timestamp list and indices to split raw data
    into sequence lists.
    """    
    with open(json_path) as f:
        parsed = json.load(f)    
    data = parsed['images']

    LLA = []
    vel_list = []
    time_list = []
    sequence = []
    indices = []
    failed_ind = []
    prev_seq = None

    for idx, image in enumerate(data): 
        img_path = image['image_path']
        path_split = img_path.split('/')   
        curr_seq = path_split[3]
        lat = image["latitude"]
        lon = image["longitude"]
        time_stamp = image["time_stamp"]
        vel = image["velocity"]
        yaw_rate = image["yaw_rate"]
        vel_list.append([vel, yaw_rate])
        time_list.append(time_stamp)
        
        # to filter out the measurements which are not in Germany
        # Better to save idx so that I can mask it as failed measurement
        # while giving into KF
        if (lat > 60.0 or lat < 40.0) or (lon > 20.0 or lon < 2.5):
            failed_ind.append(idx)
            continue
            
        if idx == 0:
            sequence.append([lat, lon])         
                
#         if [lat,lon] not in sequence and curr_seq == prev_seq:
        if curr_seq == prev_seq:
            sequence.append([lat, lon])      
            
        if (curr_seq != prev_seq and idx > 0):       # checking the sequence folder
            LLA.append(sequence)
            indices.append(len(sequence))            
            sequence = []
            sequence.append([lat, lon])  # append first element of the new sequence            
            
        prev_seq = curr_seq
        
    LLA.append(sequence)   # adding last sequence to the list
    indices = np.cumsum(indices)
    failed_ind = np.cumsum(failed_ind)
    
    return LLA, vel_list, time_list, [indices, failed_ind]


def get_UTM(LLA):
    """
    Taking input as (latitude, longitude) and return UTM list as
    (Easting, Northing, Zone Number, Zone Letter)
    """    
    UTM_list = []
    if np.shape(LLA[0]) == (2,):
        lats = np.array(LLA).T[0]
        longs = np.array(LLA).T[1]
        UTM = utm.from_latlon(lats, longs)
        UTM_list.append(UTM)

    else:
        for lla in LLA:
            lats = np.array(lla).T[0]
            longs = np.array(lla).T[1]
            UTM = utm.from_latlon(lats, longs)
            UTM_list.append(UTM)

    return UTM_list


# def calculate_distance(coord1, coord2):
                            
#     if len(coord1) == 2 and len(coord2) == 2:
#         print('Calculating distance for given 2 locations')
#         lat1, lon1 = coord1
#         lat2, lon2 = coord2
#     elif (np.asarray(coord1) == np.asarray(coord2)).all():
#         lat1, lon1 = np.array(coord1[:-1]).T
#         lat2, lon2 = np.array(coord2[1:]).T
#     else:
#         lat1, lon1 = np.array(coord1).T
#         lat2, lon2 = np.array(coord2).T       

#     p = pi/180
#     ss = np.divide(np.cos(np.multiply((lat2-lat1), p)), 2) 
#     ss2 = np.multiply(np.cos(np.multiply(lat1, p)), np.cos(np.multiply(lat2, p)))
#     ss3 = (1 - np.cos(np.multiply((lon2-lon1), p))) / 2
#     a = 0.5 - ss + np.multiply(ss2, ss3)

#     distances = 12742 * np.arcsin(np.sqrt(a)) * 1000    # 2*R*asin.. in meters

#     return distances


def match_GPS(coords, radius=20.0):
    """
    Taking longitude latitude coordinates and snap them onto the
    nearest road segment.
    """
    client = osrm.Client(host='http://10.79.23.17:5000')
    if len(coords) == 1:
        response = client.nearest(
            coordinates=coords,
        #     timestamps=stamps,                
            radiuses=[radius]*len(coords)
        )
        loc = response['waypoints'][0]['location']
        locs_map = [list(reversed(loc))]
        matchings_map = locs_map        
    
    else:
        response = client.match(
            coordinates=coords,
        #     timestamps=stamps,
            overview=osrm.overview.full,
            radiuses=[radius]*len(coords)
        )

        locs = [idx['location'] for idx in response['tracepoints'] if idx]
        locs_map = [list(reversed(loc)) for loc in locs]
        matchings = response['matchings'][0]['geometry']['coordinates']
        matchings_map = [list(reversed(loc)) for loc in matchings]
   
    return locs_map, matchings_map   
    

def mark_map(loc, grouped=False, show_popup=False, classes=False):
    """
    Visualization of the positions of the traffic lights,
    and vehicle. OSM France map is used since it has max 
    zoom 20 instead of 19. 
    'grouped' is used to visualize sequences. 
    'classes' is used for final map with class information.
    """
    color_list = ['black', 'blue', 'green', 'purple', 'orange', 'darkgreen',
                  'pink', 'lightred', 'black', 'lightgray', 'beige', 'lightblue']
    shape_list = sorted(glob.glob('./shapes/*'))[:3] \
                 + sorted(glob.glob('./shapes/*'))[6:]
    cls_list = []
    tile_layer = folium.TileLayer(
        tiles='https://{s}.tile.openstreetmap.fr/osmfr/{z}/{x}/{y}.png',
        attr='&copy; OpenStreetMap France | &copy; \
        <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> \
        contributors',
        zoom_start=18,
        max_zoom=20,
        name='Frmap',
        control=False,        
    )
    
    shadow = 'https://www.nicepng.com/png/full/409-4092997_lifestyle-for-mobile-shadow-png.png'
    if classes:
        cls_list = loc[1]
        loc = loc[0]

    if grouped:
        max_len = max(max(len(loc_groups) for loc_groups in loc), len(loc))
        color_list = color_list * np.ceil(max_len / len(color_list)).astype(np.int8)
        bulb_list = ['three_aspects', 'one_aspect', 'two_aspects']
        dir_list = ['back', 'front', 'left', 'right']
        gt_cls_list = [list(p) for p in itertools.product(dir_list, bulb_list)]
        
        try:
            # If multiple sequences with multiple images are given as input
            my_map = folium.Map(location=loc[0][0][0], zoom_start=18, max_zoom=20, tiles=None)
            tile_layer.add_to(my_map)  # add France map tile
            
            for seq_id, loc_groups in enumerate(loc):
                # Creating feature group for each sequence to add corresponding images
                # to the groups.
                seq_group = folium.FeatureGroup(name=str(seq_id+1), show=show_popup)
                my_map.add_child(seq_group)                
                
                for img_id ,coords in enumerate(loc_groups):
                    # Creating feature subgroups for each image to add traffic lights
                    # to the images.
                    img_group = plugins.FeatureGroupSubGroup(seq_group, 
                                                             name="seq "+str(seq_id+1)+"img "+str(img_id+1),
                                                             show=show_popup) 
                    my_map.add_child(img_group)
                    for idx, lla in enumerate(coords):
#                         curr_cls = cls_list[img_id][idx]
                        # Choosing colors for different images
                        clr_idx = gt_cls_list.index(cls_list[img_id][idx]) if cls_list else img_id 
                        popup_name = ','.join(curr_cls) if classes else str(idx)
                        
                        marker = folium.Marker(location=lla, 
                                               popup=folium.Popup(popup_name, show=True),
                                               icon=folium.map.Icon(color=color_list[clr_idx])) 
                        marker.add_to(img_group)
                        
#             my_map.add_child(MeasureControl()) # optional distance measure        
            folium.LayerControl(collapsed=False).add_to(my_map)
            
        except: 
            # If tracked TLs with class information or multiple 
            # sequences of GPS of vehicle are given
            max_len = max(max(len(coords) for coords in loc), len(loc))
            color_list = color_list * np.ceil(max_len / len(color_list)).astype(np.int8)
            my_map = folium.Map(location=loc[0][0], zoom_start=18, max_zoom=20, tiles=None)
            tile_layer.add_to(my_map)
            for group_id ,coords in enumerate(loc):
                group = folium.map.FeatureGroup(name=str(group_id+1), show=show_popup)
                for idx, lla in enumerate(coords):
                    if classes:
                        # If classes are given with coordinates,
                        # custom icons are used to visualized 
                        # the traffic lights.
                        curr_cls = cls_list[group_id][idx]
                        curr_cls = [curr_cls[1], curr_cls[0]]                        
                        clr_idx = gt_cls_list.index(curr_cls)
                        popup_name = ','.join(curr_cls)
                        # different sizes for different bulb numbers
                        icon_sizes = [(12, 24), (12, 36), (12, 12)]
                        icon = folium.features.CustomIcon(
                                shape_list[clr_idx], 
                                icon_size=icon_sizes[(clr_idx+1)%3],
                                shadow_image=shadow,
                                shadow_size=icon_sizes[(clr_idx+1)%3],
                                shadow_anchor=(-2, 17)
                        )
                        marker = folium.Marker(location=lla, 
                                               popup=folium.Popup(popup_name, show=False),
                                               icon=icon)
                        
                    else:
                        # If there is no class information, choose colors
                        # for circular markers instead of custom icons.
                        clr_idx = group_id
                        popup_name = str(idx)
                        icon = folium.map.Icon(color=color_list[clr_idx%12])
                        marker = folium.CircleMarker(location=lla, radius=8, 
                                                     fill_color=color_list[clr_idx%12], 
                                                     color='white', fill_opacity=0.75,
                                                     show=True)
            

                    marker.add_to(group)
                my_map.add_child(group)
#             my_map.add_child(MeasureControl())       
            folium.LayerControl(collapsed=False).add_to(my_map)                 
    
    elif len(loc) == 2 and isinstance(loc[0], float) == 1:  
        # only 1 coordinate is given
        my_map = folium.Map(location=loc, zoom_start=18, max_zoom=19)
        marker = folium.Marker(location=loc, 
                               popup=folium.Popup(str(1), show=False),
                               )
        marker.add_to(my_map)

    else:        
        my_map = folium.Map(location=loc[0], zoom_start=18, max_zoom=19)
        for idx, lla in enumerate(loc):
            marker = folium.Marker(location=lla, popup=folium.Popup(str(idx),
                                        show=False))
            marker.add_to(my_map)
        
    return my_map


In [ ]:
det2_smoothed_ca[7][0], TL_splitted_det2_ca[7][0]

In [ ]:
video_list = [det2_smoothed_ca[7], det2_classes_split_ca[7]]

mark_map(video_list, classes=True , grouped=True)

In [ ]:
# video_list = [det2_smoothed_ca[0], det2_classes_split_ca[0]]

# mark_map(video_list, classes=True , grouped=True)
mark_map(smoothed_snapped[7])


In [ ]:
idx = 7
mark_map(smth_class, classes=True, grouped=True)

In [ ]:
import pymap3d as pm

# def calculate_angle(p1, p2):
#     """"""

#     if isinstance(p1, np.ndarray):
#         p1_x = p1.T[0]
#         p1_y = p1.T[1]
#         p2_x = p2.T[0]
#         p2_y = p2.T[1]
#         dy = p2_y - p1_y
#         dx = p2_x - p1_x
#         angles = np.arctan2(dy, dx) 
        
#         return angles
    
#     else:
#         angle = np.arctan2(p2[1]-p1[1], p2[0]-p1[0])    
      
#         return angle
    

def to_enu(LLA):
    """
    Converting longitude latitude to local ENU coordinates
    using pymap3d package. Altitude is taken as 0 for all 
    sequences since the difference in altitude is negligible
    for frames in a sequence.
    """
    lat0 = [seq[0][0] for seq in LLA if len(seq[0])]
    lon0 = [seq[0][1] for seq in LLA if len(seq[0])]
    h0 = np.zeros(len(lat0))
    lats = [np.transpose(seq)[0] for seq in LLA if len(seq[0])]
    lons = [np.transpose(seq)[1] for seq in LLA if len(seq[0])]
    hs = np.zeros(len(lats))

    ENU = []
    for idx in range(len(lats)):
        e, n, u = pm.geodetic2enu(lats[idx], lons[idx], hs[idx],
                                  lat0[idx], lon0[idx], h0[idx])
        
        ENU.append(np.array([e, n, u]))             

    return ENU


def angle_from_gps(gps_seq):           
    """
    Calculates angle between two consecutive 
    ENU coordinates in the whole array of
    sequence.
    """    
    gps_x = gps_seq[0]
    gps_y = gps_seq[1]
    dy = gps_y[1:] - gps_y[:-1]
    dx = gps_x[1:] - gps_x[:-1]
    dy = np.append(dy, dy[-1])
    dx = np.append(dx, dx[-1]) 
    angles = np.arctan2(dy, dx)
    
    return angles


def project_TL(LLA, TL_splitted):
    """
    Takes coordinate measurements of the vehicle
    and sequence-wise grouped traffic light 
    positions with respect to the camera frame.
    Returns projected traffic light coordinates
    in latitude longitude, ENU, 
    """
    tl_latlon_list = []
    tl_enu_list = []
    angle_list = []
    gps_tl_list = []
    
    if len(LLA[0][0]) == 2:    # [Lat, Lon] given
        gps = to_enu(LLA)    
        lat0 = [seq[0][0] for seq in LLA if len(seq[0])]
        lon0 = [seq[0][1] for seq in LLA if len(seq[0])]
        h0 = np.zeros(len(lat0))
        hs = np.zeros(len(LLA))
        
    else:        
        gps = LLA
    
    for seq_id, (gps_seq, tl_seq) in enumerate(zip(gps, TL_splitted)):
        if len(tl_seq):
            gps_arr = np.transpose(gps_seq[:2])
            angles = angle_from_gps(gps_seq)  # using ENU coordinates         
            R = np.array([[np.cos(angles), -np.sin(angles)],
                          [np.sin(angles),  np.cos(angles)]])
            R = np.transpose(R, (2,0,1))
            
            rotated_tl = [((R[idx] @ tl_img.T).T + gps_arr[idx]).tolist()
                          for idx, tl_img in enumerate(tl_seq)]
            
            if len(gps[0]) == 4:      # UTM measurements given
                zone_number = gps_seq[-2]
                zone_letter = gps_seq[-1]
                tl_latlon = [np.transpose(utm.to_latlon(tl.T[0], tl.T[1], 
                                                        zone_number, 
                                                        zone_letter)
                                         ).tolist()
                             for tl in rotated_tl]
                
            
            else:          # ENU measurements given      
                tl_latlon = [np.transpose(
                             pm.enu2geodetic(np.transpose(tl)[0], np.transpose(tl)[1], hs[idx],
                                             lat0[seq_id], lon0[seq_id], h0[seq_id]
                                             )[:2]
                                         ).tolist()
                             for idx, tl in enumerate(rotated_tl)]
                
            tl_enu_list.append(rotated_tl)    
            tl_latlon_list.append(tl_latlon)
            angle_list.append(angles)
#             gps_tl_list.append(rotated_tl)
    
        else:
            tl_enu_list.append([[]])           
            tl_latlon_list.append([[]])
            angle_list.append([[]])
#             gps_tl_list.append([[]])
            
    return tl_latlon_list, tl_enu_list, angle_list #, gps_tl_list


In [ ]:
args = argparse.ArgumentParser()

# args.label_file = '/DTLD/detectron2/imsize_640_mini_p4only_4stride/DTLD_test_inference_results.json'
# args.label_file = '/simsek/ScaledYOLOv4/runs/exp44_yolov4-csp-bulb-dir-all-multGPU-cspweights/weights/best_yolov4-csp-bulb-dir-all-multGPU-cspweights.json'
args.calib_dir = '/dtld_parsing/calibration/'
args.data_base_dir = '/DTLD/Berlin_disp'
args.images = None
# args.label_file = '/DTLD/detectron2/full_size_mini/DTLD_test_inference_results_feb.json'
# TL_coords_det2, det2_classes = positions_3D(args)
# args.label_file = '/DTLD/detectron2/R50_fullsize_customanchor/2022_02_23_2002_DTLD_test_inference_results_best.json'
# args.label_file = '/DTLD/detectron2/2022_04_07_2044_default_inp_detection.json'
# args.label_file = '/DTLD/detectron2/R50_30_130/2022_04_07_2158_DTLD_test_inference_results_best.json'
args.label_file = '/DTLD/2022_04_11_1351_berlin1_inp_detection.json'
TL_coords_det2_ca, det2_classes_ca = positions_3D(args)
# args.label_file = '/DTLD/v2.0/v2.0/DTLD_test.json'
# TL_coords_gt, gt_classes = positions_3D(args)


In [ ]:
path = '/DTLD/v2.0/v2.0/DTLD_test.json'
LLA, vel_list, times, indices = get_GPS(path)
LLA_rev = [[list(reversed(loc)) for loc in seq] for seq in LLA]
snapped = []
for idx, seq in enumerate(LLA_rev):    
    locs_map, _ = match_GPS(seq)
    snapped.append(locs_map)


In [ ]:
# snapped_utm = get_UTM(snapped)
# TL_splitted_det2 = np.split(TL_coords_det2, indices[0])
TL_splitted_det2_ca = np.split(TL_coords_det2_ca, indices[0])
# TL_splitted_gt = np.split(TL_coords_gt, indices[0])
# tl_latlon, angles, utm_tl_list = project_TL(snapped_utm, TL_splitted)
# _, _, angles = project_TL(snapped, TL_splitted_det2_ca)
# tl_latlon_gt, tl_enu_gt, gtangles = project_TL(snapped, TL_splitted_gt)
# tl_smoothed_gt, _ = project_TL(smoothed_LLA, TL_splitted)



In [ ]:
# map_groups_full_gt = [[[snap] + tl + tl_gt for snap, tl, tl_gt in zip(seq, tl_seq, tl_seq_gt) if len(tl)] 
#                        for seq, tl_seq, tl_seq_gt in zip(snapped, tl_latlon_fullsize, tl_latlon_gt)]
# map_groups_full = [[[snap] + tl for snap, tl in zip(seq, tl_seq) if len(tl)] 
#                      for seq, tl_seq in zip(snapped, tl_latlon_fullsize)]


# KF for Car to improve GPS

In [ ]:
def vel2velxy(gps_angle_init, vel_list, split_ids):
    dt = 0.66
    seq_idx = split_ids
#     enu = to_enu(snapped)
#     gps_angle_init = [angle_from_gps(seq)[0] for seq in enu]
#     gps_angle_init = [seq[0] for seq in snapped]    
    yaw_rates = np.transpose(vel_list)[1]
    yaw_rates_splitted = np.split(yaw_rates, seq_idx)
    theta_splitted = []
    for seq, theta0 in zip(yaw_rates_splitted, gps_angle_init):
        theta_seq = []
        theta_seq.append(theta0)
        for idx, yaw_rate in enumerate(seq):    
            theta_seq.append(theta_seq[idx] + yaw_rate * dt)
        theta_seq.pop(-1)
        theta_splitted.append(theta_seq)  

    velocities = np.transpose(vel_list)[0]
    vel_splitted = np.split(velocities, seq_idx)
    vel_xy_splitted = [vel_seq[...,None] * np.array([np.cos(theta_seq), 
                                                     np.sin(theta_seq)]).T 
                       for vel_seq, theta_seq in zip(vel_splitted, theta_splitted)]
    
    return [vel_xy_splitted, yaw_rates_splitted]

In [ ]:
def vel2velxy_(angles, vel_list, split_ids):
    """
    Using angles calculated before to initialize
    first angles of the sequences, and project 
    velocity of the car into East and North 
    directions by calculating the angles from
    the yaw rate information in the dataset. 
    It also splits the resulting velocity 
    lists in sequences.
    """
    dt = 0.66
    seq_idx = split_ids
    gps_angle_init = [seq[0] for seq in angles]
    yaw_rates = np.transpose(vel_list)[1]
    yaw_rates_splitted = np.split(yaw_rates, seq_idx)
    theta_splitted = []
    for seq, theta0 in zip(yaw_rates_splitted, gps_angle_init):
        theta_seq = []
        theta_seq.append(theta0)
        
        for idx, yaw_rate in enumerate(seq):    
            theta_seq.append(theta_seq[idx] + yaw_rate * dt)
            
        theta_seq.pop(-1)
        theta_splitted.append(theta_seq)  

    velocities = np.transpose(vel_list)[0]
    vel_splitted = np.split(velocities, seq_idx)
    vel_xy_splitted = [vel_seq[..., None] * np.array([np.cos(theta_seq), 
                                                     np.sin(theta_seq)]).T 
                       for vel_seq, theta_seq in zip(vel_splitted, theta_splitted)]
    
    return [vel_xy_splitted, yaw_rates_splitted]


In [ ]:
len(gps_angle_init)

In [ ]:
enu = to_enu(snapped)
gps_angle_init = [angle_from_gps(seq)[0] for seq in enu]

In [ ]:
vel_splitted = vel2velxy(gps_angle_init, vel_list, indices[0])
smoothed_snapped = kf_car(snapped, vel_splitted)

In [ ]:
import pymap3d as pm
def get_meas_kf(snapped, vel_splitted):
    """
    Constructing measurement vector for 
    Kalman Filter which is in the form of
    [x, x_dot, y, y_dot, theta_dot] 
    from the ENU coordinates, and splitted
    and projected velocity data.
    """
    lat0 = [seq[0][0] for seq in snapped]
    lon0 = [seq[0][1] for seq in snapped]
    h0 = np.zeros(len(lat0))
    lats = [np.transpose(seq)[0] for seq in snapped]
    lons = [np.transpose(seq)[1] for seq in snapped]
    hs = np.zeros(len(lats))
    xe, yn, zu = [], [], []

    for idx in range(len(lats)):
        e, n, u = pm.geodetic2enu(lats[idx], lons[idx], hs[idx],
                                  lat0[idx], lon0[idx], h0[idx])
        xe.append(e), yn.append(n), zu.append(u)  

    # measurements = [
    #     [[utm[0], vel_xy[0], utm[1], vel_xy[1], yaw_rate] 
    #      for utm, vel_xy, yaw_rate   
    #      in zip(np.transpose(utm_seq[:2]), vel_xy_seq, yaw_rate_seq)] 
    #      for utm_seq, vel_xy_seq, yaw_rate_seq 
    #      in zip(snapped_utm, vel_xy_splitted, yaw_rates_splitted)
    #     ]
    vel_xy_splitted = vel_splitted[0]
    yaw_rates_splitted = vel_splitted[1]    
    measurements_enu = [
         np.array(
         [np.asarray([xe, vel_xy[0], yn, vel_xy[1], yaw_rate])
         for xe, yn, vel_xy, yaw_rate   
         in zip(xe_seq, yn_seq, vel_xy_seq, yaw_rate_seq)]
                 ) 
         for xe_seq, yn_seq, vel_xy_seq, yaw_rate_seq 
         in zip(xe, yn, vel_xy_splitted, yaw_rates_splitted)
        ]

    meas_kf = np.array(measurements_enu)
    return meas_kf, [lat0, lon0, h0]

In [ ]:
from pykalman import KalmanFilter
import numpy as np

def kf_car(snapped, vel_splitted, plot=False):
    """
    Kalman Filter with Expectation-Maximization
    algorithm for better localization of the 
    vehicle. Snapped GPS measurments, splitted 
    and projected velocity data is converted
    KF measurements with get_meas_kf().
    """    
    dt = 0.66  # calculated from the timestamps
    
    # states = x x_dot y y_dot theta theta_dot
    transition_matrix = [[1, dt, 0, 0, 0, 0],
                         [0, 1, 0, 0, 0, 0],
                         [0, 0, 1, dt, 0, 0],
                         [0, 0, 0, 1, 0, 0],
                         [0, 0, 0, 0, 1, dt],
                         [0, 0, 0, 0, 0, 1]]
    
    # observed states = x x_dot y y_dot theta_dot
    observation_matrix = [[1, 0, 0, 0, 0, 0],
                          [0, 1, 0, 0, 0, 0],
                          [0, 0, 1, 0, 0, 0],
                          [0, 0, 0, 1, 0, 0],                     
                          [0, 0, 0, 0, 0, 1]]
   
    # get the measurements in appropriate format
    meas_kf, lla = get_meas_kf(snapped, vel_splitted)
    
    # initial localization for each sequence
    lat0, lon0, h0 = lla[0], lla[1], lla[2]    
    smoothed_LLA = []
    
    for idx, meas_seq in enumerate(meas_kf):
        if len(meas_seq):
            
            # Initializing states 
            initial_state_mean = [0.0,
                                  meas_seq[0][1],
                                  0.0,
                                  meas_seq[0][3],
                                  0.0,
                                  meas_seq[0][-1]]
            
            kf1 = KalmanFilter(transition_matrices=transition_matrix,
                               observation_matrices=observation_matrix,
                               initial_state_mean=initial_state_mean,
                               n_dim_obs=5)
            
            # Expectation-Maximization with 3 iterations on observation covariance
            kf1 = kf1.em(meas_seq, 
                         n_iter=3, 
                         em_vars=['observation_covariance'])
            
            (smoothed_state_means, smoothed_state_covariances) = kf1.filter(meas_seq)
                
            # plot the smoothed states and state covariances 
            if plot:    

                %matplotlib inline
                import matplotlib                
                import matplotlib.pyplot as plt
                matplotlib.rcParams['figure.dpi'] = 600                
                
                times = range(meas_seq.shape[0])
                fig, axs = plt.subplots(2)
                fig.suptitle('vs Time and XY Graphs')
                axs[0].plot(times, meas_seq[:, 0], 'bo',
#                             times, meas_seq[:, 2], 'ro',
#                             times, meas_seq[:, 4], 'go',         
                            times, smoothed_state_means[:, 0], 'ro',
#                             times, smoothed_state_means[:, 2], 'r--',
#                             times, smoothed_state_means[:, 4], 'g--',
                           )
                
                for i, txt in enumerate(meas_seq[:, 0]):
                    axs[0].annotate(i, (times[i], txt+1), fontsize=5)
                    axs[0].annotate(i, 
                                    (times[i], smoothed_state_means[:, 0][i]), 
                                    fontsize=5)
                    
                axs[1].plot(meas_seq[:, 0], meas_seq[:, 2], 'bo')
                axs[1].plot(smoothed_state_means[:, 0], smoothed_state_means[:, 2], 'ro')
                for i, txt in enumerate(meas_seq[:, 0]):
#                     axs[1].annotate(round(txt), (times[i], txt+1), fontsize=5)
                    axs[1].annotate(
                        i, 
                        (smoothed_state_means[:, 0][i], smoothed_state_means[:, 2][i]-1.2), 
                        fontsize=5
                    )
                
                plt.show()
#                 _ = input('Press enter to continue')

            lat_smooth, lon_smooth, h_smooth = pm.enu2geodetic(
                smoothed_state_means[1:, 0], 
                smoothed_state_means[1:, 2], 
                smoothed_state_means[1:, 4],
                lat0[idx], lon0[idx], h0[idx]
            )        
            lat_smooth = np.concatenate(([lat0[idx]], lat_smooth)) 
            lon_smooth = np.concatenate(([lon0[idx]], lon_smooth)) 
            smoothed_LLA.append(np.column_stack((lat_smooth, lon_smooth)).tolist())
        else:
            smoothed_LLA.append([[]])
                            
    return smoothed_LLA

In [ ]:
# smoothed_LLA = kf_car(snapped, vel_splitted, True)

In [ ]:
# LLA_new, _, _, _ = get_GPS(path)
# to_pop = [(indices[0] > idx).nonzero()[0][0] - (1 if idx > indices[0][0] else 0)
#           for idx in aq]
# remainder = np.array(aq) % np.array(indices[0][to_pop])
# _ = [LLA_new[pop_id].pop(rem-1) for pop_id, rem in zip(to_pop, remainder)]

In [ ]:
# smoothed_gt = [[[snap] + tl  for snap, tl in zip(seq, tl_seq)] 
#                  for seq, tl_seq in zip(smoothed_LLA, tl_smoothed_gt)]
# non_smooth_gt = [[[snap] + tl  for snap, tl in zip(seq, tl_seq)] 
#                   for seq, tl_seq in zip(snapped, tl_latlon_gt)]
# det2_smoothed_LLA = [[[snap] + tl  for snap, tl in zip(seq, tl_seq)] 
#                       for seq, tl_seq in zip(smoothed_LLA, det2_smoothed)]
# seq_id = 0
# mark_map(det2_smoothed_LLA[seq_id] + smoothed_gt[seq_id] + non_smooth_gt[seq_id], grouped=True)

# Filtering the Traffic Lights

In [ ]:
import numpy as np
from pykalman import KalmanFilter
from scipy.optimize import linear_sum_assignment


class Track(object):
    """Track class for every object to be tracked."""

    def __init__(self, prediction, attributes, trackIdCount, first_seen):
        """
        prediction: predicted xy of the object 
        attributes: classes from the model
        trackIdCount: id for each track
        first_seen: the first frame when the detection
                    is declared as a track
        """
        self.track_id = trackIdCount 
        # states are [x, y] <=> [E, N], both are measured
        transition_matrix = [[1, 0], 
                             [0, 1]]    
        observation_matrix = [[1, 0],
                              [0, 1]] 
        
        # first element of the trace of the track is initial state
        initial_state_mean = [prediction[0],
                              prediction[1]] 
        
        # initialazing observation covariance matrix
        observation_covariance = np.diag((10.0, 5.0))        
        # initialazing state covariance matrix
        state_covar_init = np.diag((30.0, 15.0))        
        self.KF = KalmanFilter(transition_matrices=transition_matrix,
                               observation_matrices=observation_matrix,
                               initial_state_mean=initial_state_mean,
                               observation_covariance=observation_covariance,
                               n_dim_obs=2)  
        # keeping last state and covariance information
        self.last_mean = initial_state_mean
        self.last_covar = state_covar_init        
        self.prediction = np.asarray(prediction)  # predicted centroids (x,y)
        self.skipped_frames = 0  # number of frames skipped undetected
        self.trace = []  # trace path of the track
        self.att = np.array(attributes)  # two-labeled class information
        self.P_list = []  # keeping state covariance matrix
        self.P_list.append(state_covar_init)
        self.trace.append(self.prediction)  # constructing trace of the track
        self.first_seen = first_seen

class Tracker(object):
    """
    Tracker class that updates track-related vectors 
    of the objects which are tracked.
    """

    def __init__(self, dist_thresh, max_frames_to_skip):
        """
        dist_thresh: distance threshold. When exceeds the threshold,
                     track will be deleted and new track is created.
        max_frames_to_skip: maximum allowed frames to be skipped for
                            the track undetected.
        min_trace_length
        """
        self.dist_thresh = dist_thresh
        self.max_frames_to_skip = max_frames_to_skip
        self.tracks = []  
        self.trackIdCount = 0  # trackIdCount: identification of each track
    
#     def group(self, detections, classes):
#         cls = np.array(classes)
#         try:
#             srt = np.lexsort((cls[:,1], cls[:,0]))
#         except:
# #             pdb.set_trace()
#             return detections, classes
#         cls_srt= np.array(cls)[srt]
#         grouped = []
#         cls_grp = []
#         for key, group in itertools.groupby(cls_srt, lambda x: (x[0], x[1])):
#             group = list(group)
#             cls_grp.append(group)
#             grouped.append(len(group))

#         group_ind = np.cumsum(grouped)
#         det_srt = np.array(detections)[srt]    
#         det_grp = np.split(det_srt, group_ind[:-1])
#         det_list = []

#         for idx, dets in enumerate(det_grp):
#             if len(dets) == 1:
#                 det_list.append(dets)
#                 continue
#             pwise = pdist(dets)
#             close_dets = np.argwhere(squareform(pwise) < 5.0)
#             couple_ind = np.argwhere(np.diff(close_dets)[:, 0] > 0)
#             if couple_ind.size:
#                 same_box_ind = couple_ind[:, 0] 
#                 same_dets = dets[close_dets[same_box_ind]]
#                 same_dets = np.concatenate([] + same_dets[:].tolist(), axis=0)
#                 mean_det = np.mean(same_dets, axis=0)
#                 other_dets = np.delete(dets, close_dets[same_box_ind], axis=0)
#                 cls_grp[idx] = np.delete(cls_grp[idx], close_dets[same_box_ind][:, -1], axis=0)
#                 det_list.append(np.concatenate([mean_det[None,:], other_dets], axis=0))

#             else:
#                 det_list.append(dets)
            
#         detections = np.concatenate(det_list[:0] + det_list[:], axis=0)
#         classes = np.concatenate(cls_grp[:0] + cls_grp[:], axis=0)
            
#         return detections, classes
                
    def calculate_cost(self, detections, classes):
        """
        Calculating the Euclidean distance between the
        detections and the tracks with the same class
        information, and constructing cost matrix. 
        """
        N = len(self.tracks)
        M = len(detections)
        self.cost = np.zeros((N, M))
        for track_id, track in enumerate(self.tracks):
            for det_id, detection in enumerate(detections):
                if all(track.att == classes[det_id]):
                    # same class information 
                    diff = track.prediction - np.array(detection)
                    # Euclidean distance
                    distance = np.sqrt(diff.data[0]**2
                                       + diff.data[1]**2) 
                    self.cost[track_id][det_id] = distance
                else: 
                    # classes are not the same, put high cost
                    self.cost[track_id][det_id] = 1e8
                
    def hungarian(self, detections, classes, det_idx):
        """
        Deploying Hungarian Algorithm to assign the detections
        to the correct tracks. 
        """
        N = len(self.tracks)
        assignment = [-1] * N  # initializing with no matches
        # Hungarian Algorithm
        row_ind, col_ind = linear_sum_assignment(self.cost)
        for i in range(len(row_ind)):
            assignment[row_ind[i]] = col_ind[i]

        # Identify tracks with no assignment, if any
        for i in range(len(assignment)):
            if assignment[i] != -1:
                # check for cost distance threshold.
                # If cost is very high then un_assign the track
                if (self.cost[i][assignment[i]] > self.dist_thresh):
                    assignment[i] = -1

        # If tracks are not detected for a long time, remove them
        del_tracks = []
        for i in range(len(self.tracks)):
            if self.tracks[i].skipped_frames >= self.max_frames_to_skip:
                del_tracks.append(i)                

        if len(del_tracks) > 0:  # only when skipped frame exceeds max
            self.tracks = np.delete(self.tracks, del_tracks, axis=0).tolist()
            assignment = np.delete(assignment, del_tracks, axis=0).tolist()
        
        # Looking for unassigned detections
        for i in range(len(detections)):
            if (i not in assignment):
                # create new track for unassigned detections
                track = Track(detections[i], 
                              classes[i],
                              self.trackIdCount,                              
                              det_idx)
                self.trackIdCount +=1
                self.tracks.append(track)        

        return assignment

    def update(self, detections, det_idx):
        """
        - Initialize tracks
        - Calculate cost between predicted and
          detected centroids
        - Assign detections to tracks using Hungarian
          Algorithm
        - Kalman Filter update step
        detections: model predictions in the form of
                    [xy, classes].        
        det_idx: it is used as the first seen frame        
        """
        classes = detections[1]
        detections = detections[0]
        
        # Create tracks if no tracks vector found
        if (len(self.tracks) == 0):          
            for i in range(len(detections)):
                track = Track(detections[i], classes[i], 
                              self.trackIdCount, 
                              det_idx)                
                self.trackIdCount += 1
                self.tracks.append(track)
        
        # No need for any action at first frame
        # of the sequence.
        if det_idx == 0:
            return
        
        # calculate the cost using both xy and classes
        self.calculate_cost(detections, classes)
        
        # assigning detections to tracks with Hungarian Algorithm,
        # or creating new tracks.        
        assignment = self.hungarian(detections, classes, det_idx)
        
        for i in range(len(assignment)):
            mean_curr = self.tracks[i].last_mean
            covar_curr = self.tracks[i].last_covar            
            if assignment[i] != -1:
                # since there is assignment, set skipped frames to 0
                self.tracks[i].skipped_frames = 0
                
                mean_curr, covar_curr = self.tracks[i].KF.filter_update(
                    mean_curr, 
                    covar_curr, 
                    detections[assignment[i]],
                )
                
                mean_curr = mean_curr.data  # data of masked array 
            else:
                # use covar_curr as observation covariance since there is no 
                # observation
                mean_curr, covar_curr = self.tracks[i].KF.filter_update(
                    mean_curr, 
                    covar_curr,
                    observation=None,
                    observation_covariance=covar_curr
                )
                # since no assignment for the track, it is skipped
                self.tracks[i].skipped_frames += 1                
           
            self.tracks[i].prediction = mean_curr
            self.tracks[i].trace.append(self.tracks[i].prediction)                
            self.tracks[i].last_mean = mean_curr
            self.tracks[i].last_covar = covar_curr
            self.tracks[i].P_list.append(covar_curr)            


In [ ]:
track_results_ca = tl_tracker(filtered_smoothed_ca, smoothed_LLA)

In [ ]:
from copy import deepcopy

def thres_filter(smoothed, heading_angles, thres):
    """
    
    """
    img_mean_list = [np.mean(img, axis=0) for seq in smoothed[0] for img in seq]
    seq_mean_list = np.split(img_mean_list, indices[0]) #[indices[0]<151]) # 150 image for now
    filtered_smooth = deepcopy(smoothed[0])
    classes = deepcopy(smoothed[1])
    if [[]] in filtered_smooth:
        filtered_smooth = [seq for seq in filtered_smooth if seq[0]] 
        classes = [seq for seq in classes if seq.size]
    
    for seq_id, (seq_mean, tl_seq, angle) in enumerate(zip(seq_mean_list, filtered_smooth, heading_angles)):
        R = np.array([[np.cos(angle), -np.sin(angle)],
                      [np.sin(angle),  np.cos(angle)]])
        R = np.transpose(R, (2, 0, 1))

        for img_id, (img_mean, tl_img) in enumerate(zip(seq_mean, tl_seq)):            
            diff = tl_img - img_mean
            diff = (R[img_id] @ diff.T).T   # projecting according to heading angle of the car
            dist2mean = np.sqrt(diff.T[0] ** 2 + 10 * diff.T[1] ** 2).T   # elliptic window 
#             if img_id == 12:
#                 import pdb;pdb.set_trace()
            if (dist2mean > thres).any():
                del_id = np.nonzero(dist2mean > thres)
                filtered_smooth[seq_id][img_id] = np.delete(filtered_smooth[seq_id][img_id], 
                                                            del_id, axis=0).tolist()
                classes[seq_id][img_id] = np.delete(classes[seq_id][img_id], 
                                                    del_id, axis=0).tolist()
    return [filtered_smooth, classes], seq_mean_list


In [ ]:
def tl_tracker(filtered_smoothed, smoothed_LLA):
    """
    filtered_smoothed: [xy, classes] of the projected 
                       traffic lights
    smoothed_LLA: latitude longitude coordinates of the 
                  vehicle to get initial position for
                  each sequence
    """
    results = []
    trace_list = []
    att_list = []
    covar_list = []
    first_seen_list = []
    del_traces_list, del_covars_list, del_seen_list = [], [], []
    
    coords = filtered_smoothed[0]
    classes = filtered_smoothed[1]
    # a traffic light should be detected atleast 2 frames 
    min_trace_len = 2  
    search_dist = 10.0  # distance threshold between the lights
    
    # remove empty ones due to missing disparity images
    if [[]] in coords:
        coords = [seq for seq in coords if seq[0]]            
    
    for seq_id, det_seq in enumerate(coords):
        # set allowed skipped frames for the sequence 
        max_frames_to_skip = np.ceil(len(det_seq) / 3).astype(np.int8)
        # initialize the Tracker for the sequence
        tracker = Tracker(search_dist, max_frames_to_skip) 
        # get initial position of the vehicle to set it as the origin of
        # local ENU frame
        lat0 = smoothed_LLA[seq_id][0][0]
        lon0 = smoothed_LLA[seq_id][0][1]
        h0 = 0

        for det_idx, det_img in enumerate(det_seq):
            # update Tracker for each frame
            tracker.update([det_img, 
                            classes[seq_id][det_idx]], 
                            det_idx)
#             else:
#                 continue
        
        tracks = tracker.tracks
        traces, att, covars, first_seen, skipped \
            = np.transpose([[track.trace, track.att, track.P_list, 
                             track.first_seen, track.skipped_frames] 
                             for track in tracks]).tolist()
        del_idx = []
        for idx, trace in enumerate(traces):
            trace = np.array(trace)
            _, index= np.unique(trace, return_index=True,
                                axis=0)
            # find the frames where an assignment occurs     
            uniq = trace[np.sort(index)]       
            if (len(uniq) <= min_trace_len): 
                del_idx.append(idx)
        
        # keep deleted vectors 
        del_traces = np.array(traces)[np.array(del_idx)].tolist()
        del_covars = np.array(covars)[np.array(del_idx)].tolist()
        del_seen = np.array(first_seen)[np.array(del_idx)].tolist()
        
        traces = np.delete(traces, del_idx, axis=0).tolist()
        att = np.delete(att, del_idx, axis=0).tolist()
        covars = np.delete(covars, del_idx, axis=0).tolist()
        first_seen = np.delete(first_seen, del_idx, axis=0).tolist()
        last_coords = [group[-1] for group in traces if group]
        if seq_id == 7:
            import pdb;pdb.set_trace()
        # group the last coordinates since different tracks can 
        # start from different positions but end up very close
        # to each other
        last_coords, att = group_last(last_coords, att)
        # transform from ENU to latitude longitue
        tl_latlon = [np.transpose(
                     pm.enu2geodetic(tl.T[0], tl.T[1], h0,
                                     lat0, lon0, h0
                                    )[:2]
                                 ).tolist()
                     for idx, tl in enumerate(last_coords)] 
        
        results.append(tl_latlon)
        trace_list.append(traces)
        att_list.append(att)
        covar_list.append(covars)
        first_seen_list.append(first_seen)
        del_traces_list.append(del_traces)
        del_covars_list.append(del_covars)
        del_seen_list.append(del_seen)
        
    return results, trace_list, att_list, covar_list, first_seen_list, \
           [del_traces_list, del_covars_list, del_seen_list]


In [ ]:
import scipy
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import fclusterdata

def group_last(detections, classes):
    """
    Groups the last coordinates of each traces if
    they are closer than 5.0 m and have same class
    information.
    """
    cls = np.array(classes)
    try:
        srt = np.lexsort((cls[:,1], cls[:,0]))
    except:
        return detections, classes
    
    cls_srt= np.array(cls)[srt]
    grouped = []
    cls_grp = []
    # grouping sorted classes
    for key, group in itertools.groupby(cls_srt, lambda x: (x[0], x[1])):
        group = list(group)
        cls_grp.append(group)
        grouped.append(len(group))
    
    # grouping xy of the detections according to classes
    group_ind = np.cumsum(grouped)
    det_srt = np.array(detections)[srt]    
    det_grp = np.split(det_srt, group_ind[:-1])
    det_list = []
    
    for idx, dets in enumerate(det_grp):
        if len(dets) == 1:  # only one element in the group
            det_list.append(dets[0])
            continue
        # sorting the xys in the group    
#         srt = np.lexsort((dets[:,1], dets[:,0]))
#         dets = dets[srt]  
        # clustering the xys with 5.0m distance threshold
        cluster_ind = fclusterdata(dets, 5.0, criterion='distance') 
#         split_ind = np.where(np.abs(np.diff(cluster_ind)) > 0)[0] + 1
#         det_splits = np.split(dets, split_ind)
        det_splits = [dets[cluster_ind==idx+1, :] 
                      for idx in range(max(cluster_ind ))]
        for k, split in enumerate(det_splits):
            if len(split) > 1:
                del_cnt = len(split) - 1      
                # get mean of the cluster, since they are the same object                
                split = [np.mean(split, axis=0)]
                # only keep one class information, delete others                
                cls_grp[idx] = cls_grp[idx][:-del_cnt]
            
            det_list.append(split[0])

    detections = np.concatenate([det_list], axis=0)
    classes = np.concatenate(cls_grp[:0] + cls_grp[:], axis=0)

    return detections, classes

In [ ]:
# import numpy as np
# numbers = np.array([[ 76.18599767,  69.66877349],
#        [ 75.4227973 ,  68.72071791],
#        [ 91.81466867,  65.56888397],
#        [ 75.51092048,  68.86807941],
#        [106.55387751,  91.43347366],
#        [ 92.27926526,  65.3441042 ],
#        [ 76.38748013,  69.46072411],
#        [ 94.31703173,  86.13376495],
#        [126.56059911, 112.17138298],
#        [198.47744585, 179.00238334],
#        [132.24123637, 106.0019046 ],
#        [159.55315234, 119.94404613],
#        [124.50811689, 110.62058564],
#        [162.14055616, 148.44267404],
#        [105.88063973,  75.14234612],
#        [123.82607071, 109.80604143],
#        [133.97161392,  98.73633552],
#        [126.41158668, 106.48116768]])

# srt = np.lexsort((numbers[:,1], numbers[:,0]))
# numbers = numbers[srt]

In [ ]:
# vel_splitted = vel2velxy_(angles, vel_list, indices[0])
# smoothed_LLA = kf_car(snapped, vel_splitted)

In [ ]:
# tl_smoothed_gt, smoothed_enu, _ = project_TL(smoothed_LLA, TL_splitted_gt)
# det2_smoothed, det2_smoothed_enu, det2_angles = project_TL(smoothed_LLA, TL_splitted_det2)
# det2_smoothed_ca, det2_smoothed_enu_ca, det2_angles_ca = project_TL(smoothed_LLA, TL_splitted_det2_ca)

In [ ]:
det2_smoothed_ca, det2_smoothed_enu_ca, det2_angles_ca = project_TL(smoothed_snapped, TL_splitted_det2_ca)

In [ ]:
# det2_ca, det2_enu_ca, det2_ang_ca = project_TL(snapped, TL_splitted_det2_ca)

In [ ]:
# det2_classes_split = np.split(det2_classes, indices[0])
# det2_smoothed_preds = [det2_smoothed_enu, det2_classes_split]
# filtered_smoothed, seq_mean_list = thres_filter(det2_smoothed_preds, det2_angles, 150.0)

det2_classes_split_ca = np.split(det2_classes_ca, indices[0])
det2_smoothed_preds_ca = [det2_smoothed_enu_ca, det2_classes_split_ca]
filtered_smoothed_ca, seq_mean_list_ca = thres_filter(det2_smoothed_preds_ca, det2_angles_ca, 150.0)

# Start tracking

In [ ]:
filtered_smoothed_ca = det2_smoothed_preds_ca

In [ ]:
# track_results = tl_tracker(filtered_smoothed, smoothed_LLA)
# res = track_results[0]
# traces = track_results[1]
# classes = track_results[2]
# covar_list = track_results[3]
# first_seen_list = track_results[4]
# res_class = [res, classes]

track_results_ca = tl_tracker(filtered_smoothed_ca, smoothed_LLA)
# track_results_ca n= tl_tracker(filtered_smoothed_ca, snapped)

# res = track_results_ca[0]
res_smooth = track_results_ca[0]
traces = track_results_ca[1]
classes = track_results_ca[2]
covar_list = track_results_ca[3]
first_seen_list = track_results_ca[4]
del_traces = track_results_ca[5][0]
del_covars = track_results_ca[5][1]
del_seen = track_results_ca[5][2]



In [ ]:
clstr = np.array([2, 2, 1, 2, 1, 2])
detss = np.array([[119.22331307,   6.22576122],
                   [120.66184177,   4.10079783],
                   [125.36069431,  19.6902085 ],
                   [125.52098911,   4.81679856],
                   [125.82385457,  19.63153604],
                   [128.41290565,   5.05028084]])


In [ ]:
[detss[clstr==idx+1, :] for idx in range(max(clstr))]

In [ ]:
smth_class = [res_smooth, classes]
mark_map(smth_class, classes=True, grouped=True)

In [ ]:
traces[7], classes[7]

In [ ]:
mark_map(det2_smoothed_ca[12] + [res_smooth[12]], grouped=True)

# COVARIANCE ETC.

In [ ]:
seq13_traces = traces[13]
seq13_covars = covar_list[13]
seq13_seen = first_seen_list[13]
seq13_len = len(filtered_smoothed[0][13])
poses = [[np.sqrt(loc.data[0]**2 + loc.data[1]**2)
         for loc in trace] for trace in seq13_traces]
frames = range(seq13_len)
# poses = [pose + [pose[-1]] * (len(covar) - len(pose)) 
#          for pose, covar in zip(poses, seq13_covars)]
len(res[0])

In [ ]:
seq13_deltraces = del_traces[13]
seq13_delcovars = del_covars[13]
seq13_delseen = del_seen[13]
delposes = [[np.sqrt(loc.data[0]**2 + loc.data[1]**2)
             for loc in trace] for trace in seq13_deltraces]

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
# from matplotlib.patches import Ellipse
matplotlib.rcParams['figure.dpi'] = 600

# figsize = width / float(dpi), height / float(dpi)
#     figsize = width, height

# Create a figure of the right size with one axes that takes up the full figure
# fig = plt.figure(figsize=figsize)
# ax = fig.add_axes([0, 0, 1, 1])

# ells = [
#         [Ellipse(xy=(50-(fr+seen)*2, 50-pose/3), 
#                  width=np.diag(covar)[0], 
#                  height=np.diag(covar)[1]) 
#         for fr, pose, covar in zip(frames, pose_group, covar_group)]         
#         for pose_group, covar_group, seen in zip(poses, 
#                                                  seq13_covars, 
#                                                  seq13_seen)
#        ]

# del_ells = [
#             [Ellipse(xy=(50-(fr+seen)*2, 50-pose/3), 
#                      width=np.diag(covar)[0], 
#                      height=np.diag(covar)[1]) 
#             for fr, pose, covar in zip(frames, pose_group, covar_group)]         
#             for pose_group, covar_group, seen in zip(delposes, 
#                                                      seq13_delcovars, 
#                                                      seq13_delseen)
#            ]

# plt.ion()
# plt.rcParams.update({'axes.labelsize': 'medium'})
# plt.rcParams.update({'xtick.labelsize': 'x-small'})
# plt.rcParams.update({'ytick.labelsize': 'x-small'})

# for e_trace, dele_trace in zip(ells, del_ells):
#     plt.axes()
#     plt.xlabel('Frame Number')
#     plt.ylabel('Distance from the Car at Frame 0 (m)')
#     plt.axis([0,25,0,150])
#     plt.xticks(np.arange(0, 25, 1))
#     # plt.plot(5, poses[0][0], 'ro') # against 1st x, 1st y
#     plt.grid(True, axis='x', alpha=0.1)

#     plt.twinx()
#     plt.ylabel('Northing (m)', fontsize=10)
#     plt.axis([0,25,50,0])
#     plt.twiny()
#     plt.xlabel('Easting (m)')
#     plt.axis([50,0,50,0])
    
#     for idx, e in enumerate(e_trace):
#         plt.gca().add_patch(e)
#         e.set_alpha(0.1)
#         plt.plot(e.center[0], e.center[1], 'ro', markersize=3) # against 1st x, 1st y
#         try:
#             de = dele_trace[idx]
#             plt.gca().add_patch(de)
#             de.set_alpha(0.1)
#             plt.plot(de.center[0], de.center[1], 'bo', markersize=3) # against 1st x, 1st y
#         except:
#             continue
    
# #     input('stop')
#     plt.show()

covar_xy = np.array([
            np.array([
            np.array([fr+seen, 
             np.diag(covar)[0], 
             np.diag(covar)[1]]) 
            for fr, pose, covar in zip(frames, pose_group, covar_group)
            ])      
            for pose_group, covar_group, seen in zip(poses, 
                                                     seq13_covars, 
                                                     seq13_seen)
                   ])

delcovar_xy = np.array([
            np.array([
            np.array([fr+seen, 
             np.diag(covar)[0], 
             np.diag(covar)[1]])  
            for fr, pose, covar in zip(frames, pose_group, covar_group)
            ])         
            for pose_group, covar_group, seen in zip(delposes, 
                                                     seq13_delcovars, 
                                                     seq13_delseen)
                       ])

plt.ion()
plt.rcParams.update({'axes.labelsize': 'medium'})
plt.rcParams.update({'xtick.labelsize': 'x-small'})
plt.rcParams.update({'ytick.labelsize': 'x-small'})
from scipy.interpolate import make_interp_spline, BSpline
size_nodet = 3
size_det = 5
marker_det = 'v'
for e_trace, dele_trace in zip(covar_xy, delcovar_xy):  # CAREFUL 
    plt.axes()
    plt.xlabel('Frame Number')
    plt.ylabel('Distance covariance ($m^2$)')
    plt.axis([-0.5,25,0,50])
    plt.xticks(np.arange(0, 25, 1))
    plt.yticks(np.arange(0, 50, 5))
    
    # plt.plot(5, poses[0][0], 'ro') # against 1st x, 1st y
    plt.grid(True, axis='x', alpha=0.1)

#     xnew = np.linspace(e_trace[:, 0].min(), e_trace[:, 0].max(), 150)
#     splx = make_interp_spline(e_trace[:, 0], e_trace[:, 1], k=3)  # type: BSpline
#     sply = make_interp_spline(e_trace[:, 0], e_trace[:, 2], k=3)
#     covarx_smooth = splx(xnew)
#     covary_smooth = sply(xnew)   
#     plt.plot(xnew, covarx_smooth, 'r', linewidth=1, 
#              label='Covariance of light 1 in East direction') # against 1st x, 1st y
#     plt.plot(xnew, covary_smooth, color=('#f4a54c'), linewidth=1, 
#              label='Covariance of light 1 in North direction') # against 1st x, 1st y
   
    plt.plot(e_trace[:, 0], e_trace[:, 1], 'r', linewidth=1, 
             label='Covariance of light 1 in East direction') # against 1st x, 1st y
    plt.plot(e_trace[:, 0], e_trace[:, 2], color=('#f4a54c'), linewidth=1, 
             label='Covariance of light 1 in North direction') # against 1st x, 1st y
    drop_idx = np.where(np.diff(e_trace[:, 1]) < 0)[0] + 1  
    plt.plot(np.delete(e_trace[:, 0], drop_idx), np.delete(e_trace[:, 1], drop_idx), 'ro', markersize=size_nodet)  
    plt.plot(e_trace[:, 0][drop_idx], e_trace[:, 1][drop_idx], 'r' + marker_det, markersize=size_det)
    plt.plot(np.delete(e_trace[:, 0], drop_idx), np.delete(e_trace[:, 2], drop_idx), 'o', color=('#f4a54c'), markersize=size_nodet) # against 1st x, 1st y
    plt.plot(e_trace[:, 0][drop_idx], e_trace[:, 2][drop_idx], marker_det, color=('#f4a54c'), markersize=size_det) # against 1st x, 1st y
    
#     plt.plot(e_trace[:, 0], e_trace[:, 1], 'ro', markersize=3) # against 1st x, 1st y
#     plt.plot(e_trace[:, 0], e_trace[:, 2], 'o', color=('#f4a54c'), markersize=3) # against 1st x, 1st y    
    
#     del_xnew = np.linspace(dele_trace[:, 0].min(), dele_trace[:, 0].max(), 150)
#     del_splx = make_interp_spline(dele_trace[:, 0], dele_trace[:, 1], k=1)  # type: BSpline
#     del_sply = make_interp_spline(dele_trace[:, 0], dele_trace[:, 2], k=2)
#     del_covarx_smooth = del_splx(del_xnew)
#     del_covary_smooth = del_sply(del_xnew) 
    
#     p3_x = np.poly1d(np.polyfit(dele_trace[:, 0],  dele_trace[:, 1], 9))
#     plt.plot(del_xnew, p3_x(del_xnew), 'r', linewidth=1, label='Covariance of light 1 in East direction') # against 1st x, 1st y
#     plt.plot(del_xnew, del_covarx_smooth, 'b', linewidth=1, label='Covariance of light 2 in East direction') # against 1st x, 1st y
#     plt.plot(del_xnew, del_covary_smooth, color=('#4a81b8'), linewidth=1, label='Covariance of light 2 in North direction') # against 1st x, 1st y
   
    plt.plot(dele_trace[:, 0], dele_trace[:, 1], 'b', linewidth=1, 
             label='Covariance of light 2 in East direction') # against 1st x, 1st y
    plt.plot(dele_trace[:, 0], dele_trace[:, 2], color=('#4a81b8'), linewidth=1, 
             label='Covariance of light 2 in North direction') # against 1st x, 1st y  
    drop_idx = np.where(np.diff(dele_trace[:, 1]) < 0)[0] + 1
    plt.plot(np.delete(dele_trace[:, 0], drop_idx), np.delete(dele_trace[:, 1], drop_idx), 'bo', markersize=size_nodet)    
    plt.plot(dele_trace[:, 0][drop_idx], dele_trace[:, 1][drop_idx], 'b' + marker_det, markersize=size_det)        
    plt.plot(np.delete(dele_trace[:, 0], drop_idx), np.delete(dele_trace[:, 2], drop_idx), 'o', color=('#4a81b8'), markersize=size_nodet) # against 1st x, 1st y
    plt.plot(dele_trace[:, 0][drop_idx], dele_trace[:, 2][drop_idx], marker_det, color=('#4a81b8'), markersize=size_det) # against 1st x, 1st y
    
    
    
    plt.legend(fontsize='x-small')   
    plt.show()
    break

In [ ]:
drop_idx = np.where(np.diff(dele_trace[:, 1]) < 0)[0] + 1
~drop_idx

In [ ]:
import matplotlib
from matplotlib.patches import Ellipse
matplotlib.rcParams['figure.dpi'] = 600
import time

# figsize = width / float(dpi), height / float(dpi)
#     figsize = width, height

# Create a figure of the right size with one axes that takes up the full figure
# fig = plt.figure(figsize=figsize)
# ax = fig.add_axes([0, 0, 1, 1])

ells = [
        [Ellipse(xy=(50-(fr+seen)*2, 50-pose/2), 
                 width=np.diag(covar)[0], 
                 height=np.diag(covar)[1]) 
        for fr, pose, covar in zip(frames, pose_group, covar_group)]         
        for pose_group, covar_group, seen in zip(poses, 
                                                 seq13_covars, 
                                                 seq13_seen)
       ] 

del_ells = [
            [Ellipse(xy=(50-(fr+seen)*2, 50-pose/3), 
                     width=np.diag(covar)[0], 
                     height=np.diag(covar)[1]) 
            for fr, pose, covar in zip(frames, pose_group, covar_group)]         
            for pose_group, covar_group, seen in zip(delposes, 
                                                     seq13_delcovars, 
                                                     seq13_delseen)
           ]

plt.ion()
plt.rcParams.update({'axes.labelsize': 'medium'})
plt.rcParams.update({'xtick.labelsize': 'x-small'})
plt.rcParams.update({'ytick.labelsize': 'x-small'})

for e_trace, dele_trace in zip(ells, del_ells): # CAREFUL
    plt.axes()
    plt.xlabel('Frame Number')
    plt.ylabel('Distance from the Car at Frame 0 (m)')
    plt.axis([-0.5,25,0,100])
    plt.xticks(np.arange(0, 25, 1))
    # plt.plot(5, poses[0][0], 'ro') # against 1st x, 1st y
    plt.grid(True, axis='x')

#     plt.twinx()
#     plt.ylabel('Northing (in meters)', fontsize=10)
#     plt.axis([0,25,50,0])
#     plt.twiny()
#     plt.xlabel('Easting (in meters)')
#     plt.axis([50,0,50,0])
    
#     for e in e_trace:
#         plt.gca().add_patch(e)
#         e.set_alpha(0.1)
    for idx, e in enumerate(e_trace):
        plt.plot((50-e.center[0])/2, (50-e.center[1])*2, 'ro', markersize=3) # against 1st x, 1st y 
        try:
            de = dele_trace[idx]
#             plt.gca().add_patch(de)
#             de.set_alpha(0.1)
            plt.plot((50-de.center[0])/2, (50-de.center[1])*2, 'bo', markersize=3) # against 1st x, 1st y
        except:
            continue
        
    plt.legend(labels=['Distances of the predicted locations of traffic light 1 from the car at frame 0', 
                       'Distances of the predicted locations of traffic light 2 from the car at frame 0'], 
               fontsize='x-small',
               loc='lower right')
    plt.show()

In [ ]:
len(del_ells)

In [ ]:
# tl_latlon_default, default_enu, _, _ = project_TL(LLA, TL_splitted_gt)

In [ ]:
# map_idx1_tracked = 
mark_map(res_class, grouped=True, classes=True)
# map_idx1_tracked.save('map_idx1_tracked_last.html')

In [ ]:
idx = 1

map_idx1_all = mark_map([det2_smoothed[idx], det2_classes_split[idx]], grouped=True, classes=True)
map_idx1_all.save('map_idx1_all.html')

In [ ]:
with open('/DTLD/v2.0/v2.0/DTLD_test.json', 'r') as f:
    parsed = json.load(f)

images = parsed['images']
prev_seq = ''
count = 0
len_seq = 0
for image in images:
    path = image['image_path']
    seq = path.split('/')[-2]
    if seq != prev_seq:
        print(len_seq)
        len_seq = 1
        count += 1
    else:
        len_seq += 1
    if count == 14:
        print(len_seq)
        print(seq)
        break
    prev_seq = seq
        
np.diff(indices[0][:15])

In [ ]:

        
#     if grouped:
#         max_len = max(max(len(loc_groups) for loc_groups in loc), len(loc))
#         color_list = color_list * np.ceil(max_len / len(color_list)).astype(np.int8)
# #         import pdb;pdb.set_trace()
#         try:
# #             isinstance(loc[0][0][0], list) or isinstance(loc[0][0][0], np.ndarray):
#             my_map = folium.Map(location=loc[0][0][0], zoom_start=18, max_zoom=19)
#             for seq_id, loc_groups in enumerate(loc):
#                 seq_group = folium.FeatureGroup(name=str(seq_id+1), show=show_popup)
#                 my_map.add_child(seq_group)                
#                 for img_id ,coords in enumerate(loc_groups):
# #                     img_group = folium.map.FeatureGroup(name=str(img_id+1), show=show_popup)
#                     img_group = plugins.FeatureGroupSubGroup(seq_group, 
#                                                              name="seq "+str(seq_id+1)+"img "+str(img_id+1),
#                                                              show=show_popup) 
#                     my_map.add_child(img_group)
#                     for idx, lla in enumerate(coords):
#                         popup_name = ','.join(cls_list[img_id][idx]) if classes else str(idx)                        
#                         marker = folium.Marker(location=lla, 
#                                                popup=folium.Popup(popup_name, show=True),
#                                                icon=folium.map.Icon(color=color_list[img_id]))        
#                         marker.add_to(img_group)                    
#             folium.LayerControl(collapsed=False).add_to(my_map)
#         except:   
#             max_len = max(max(len(coords) for coords in loc), len(loc))
#             color_list = color_list * np.ceil(max_len / len(color_list)).astype(np.int8)
#             my_map = folium.Map(location=loc[0][0], zoom_start=18, max_zoom=19)
#             for group_id ,coords in enumerate(loc):
#                 group = folium.map.FeatureGroup(name=str(group_id+1), show=show_popup)
#                 for idx, lla in enumerate(coords):
#                     popup_name = ','.join(cls_list[group_id][idx]) if classes else str(idx)
#                     marker = folium.Marker(location=lla, 
#                                            popup=folium.Popup(popup_name, show=False),
#                                            icon=folium.map.Icon(color=color_list[group_id]))        
#                     marker.add_to(group)
#                 my_map.add_child(group)
#             folium.LayerControl(collapsed=False).add_to(my_map)    

In [ ]:
# import scipy
# from scipy.spatial.distance import pdist, squareform

# classes = [['1', '5'], ['3', '5'], ['2', '4'], ['2', '4'], ['1', '5'], ['1', '5'], ['3', '5'], ['1', '5'], ['3', '5'], ['2', '5']]
# srt = np.argsort(classes, axis=0)[:,0]
# cls_srt= np.array(classes)[srt]
# grouped = []
# for key, group in itertools.groupby(cls_srt, lambda x: (x[0], x[1])):
#     grouped.append(len(list(group)))
    
# group_ind = np.cumsum(grouped)
# det_srt = np.array(detections)[srt]    
# det_grp = np.split(det_srt, group_ind[:-1])
# det_list = []

# for dets in det_grp:
#     if len(dets) == 1:
#         det_list.append(dets)
#         continue
#     pwise = pdist(dets)
#     close_dets = np.argwhere(squareform(pwise) < 15.0)
#     couple_ind = np.argwhere(np.diff(close_dets)[:, 0]>0)
#     if couple_ind.size:
#         same_box_ind = couple_ind[0][0] 
#         same_dets = dets[close_dets[same_box_ind]]
#         mean_det = np.mean(same_dets, axis=0)
#         other_dets = np.delete(dets, close_dets[same_box_ind], axis=0)      
#         det_list.append(np.concatenate([mean_det[None,:], other_dets], axis=0))
#     else:
#         det_list.append(dets) 
